# Lab 2 Presentation 1

In [1]:
import ugradio
import numpy as np

import ugradio.pico as pico
import ugradio.dft as dft
import ugradio.doppler as doppler
import ugradio.nch as nch
import ugradio.timing as timing

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, FloatSlider, Layout

import astropy
import unittest

## Jen : Gather Data

In [2]:
#cold_data_1 = np.load('../Data/big_horn_data_1000.npy.zip')
#cal_data_1 = np.load('../Data/big_horn_data_1000.npz.zip')
cold_data_2 = np.load('../Data/50mV cold data.npz')
cal_data_2 = np.load('../Data/50mV data with rain.npz')

FileNotFoundError: [Errno 2] No such file or directory: '../Data/50mV cold data.npz'

## Combine Real and Complex Signal Components

In [ ]:
#cold_data = cold_data_1['big_horn_data_1000'][0] - (cold_data_1["big_horn_data_1000"][1]*5/4*1j)
#cal_data = cal_data_1['big_horn_data_1000'][0] - (cal_data_1["big_horn_data_1000"][1]*5/4*1j)
cold_data = cold_data_2['arr_0'][0] - (cold_data_2["arr_0"][1]*5/4*1j) #SKY
cal_data = cal_data_2['arr_0'][0] - (cal_data_2["arr_0"][1]*5/4*1j) #RAIN 
#cold_data = galactic_data[0] - (galactic_data[1]*5/4*1j)

## Resize Blocks

In [ ]:
# resize blocks
cold_data.shape = (-1, 2000)
cal_data.shape = (-1, 2000)

In [ ]:
cal_data.shape

In [ ]:
cold_data.shape

## Plot Raw Horn Data

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(cold_data[0]/2**16, "mediumpurple", label="Cold")
plt.plot(cal_data[0]/2**16, "dodgerblue", label="Blackbody")
plt.xlabel("Samples", fontsize=15)
plt.ylabel("Voltage (V)", fontsize=15)
plt.title("Cold and Blackbody Horn Data", fontsize=25)
plt.legend(fontsize=15)

## Olivia : Calculate Average Power Spectra of s_cold 

In [ ]:
# COLD DATA, SKY DATA, S_OFF BECAUSE WE DID NOT OBSERVE HYDROGEN IN THIS CAPTURE
freq = np.fft.fftfreq(cold_data.shape[1], 1/62.5e6)
freq = np.fft.fftshift(freq)/1e6

blocks = 8000
avg_power_array = []
for n in range (0, blocks):
    data_new = cold_data[n]
    ft = np.fft.fft(data_new)
    ft = np.fft.fftshift(ft)
    power = abs(ft)**2
    avg_power_array.append(power)
    
avg_power_array = np.mean(avg_power_array, axis = 0)
s_cold = avg_power_array

## Plot Average Power Spectra of s_cold

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(freq + 1230 + 190, avg_power_array, "mediumpurple", linewidth=2)
plt.yscale("log")
plt.xlabel("Frequency (MHz)", fontsize=15)
plt.ylabel("Power", fontsize=15)
plt.title("Average Power Spectrum of Cold Data", fontsize=25)
plt.legend(fontsize=15)

#plt.xlim(-2, 2)
#plt.vlines(1420.4058, ymin=10e8, ymax=10e11, linewidth=.5)

## Calculate Average Power Spectra of s_cal

In [ ]:
# CAL DATA, RAIN DATA, WHICH IS S_ON BECAUSE WE DO SEE THE HYDROGEN LINE
cal_freq = np.fft.fftfreq(cal_data.shape[1], 1/62.5e6)
cal_freq = np.fft.fftshift(cal_freq)/1e6

blocks = 800
avg_rain_power_array = []
for n in range (0, blocks):
    data_new = cal_data[n]
    ft = np.fft.fft(data_new)
    ft = np.fft.fftshift(ft)
    power = abs(ft)**2
    avg_rain_power_array.append(power)
    
avg_rain_power_array = np.mean(avg_rain_power_array, axis=0)
s_cal = avg_rain_power_array

## Plot Average Power Spectra of s_cal

In [ ]:
#%matplotlib notebook
plt.figure(figsize=(15,8))
plt.plot(cal_freq + 1230 + 190, avg_rain_power_array, "dodgerblue", linewidth=2)
plt.yscale("log")
plt.xlabel("Frequency (MHz)", fontsize=15)
plt.ylabel("Power", fontsize=15)
plt.title("Average Power Spectrum of Blackbody Data", fontsize=25)
plt.legend(fontsize=15)
#plt.xlim(-1, 1)
#plt.vlines(1420.4058, ymin=10e8, ymax=10e11, linewidth=.5)

In [ ]:
#%matplotlib notebook

plt.figure(figsize=(15,8))
plt.plot(freq + 1230 + 190, avg_power_array, "mediumpurple", linewidth=3, label="cold")
plt.plot(cal_freq + 1230 + 190, avg_rain_power_array, "dodgerblue", linewidth=2, label="blackbody")
plt.yscale("log")
plt.xlabel("Frequency (MHz)", fontsize=15)
plt.ylabel("Power", fontsize=15)
plt.title("Overlapping Average Power Spectra of Calibration Data", fontsize=25)
plt.legend(fontsize=15)
#plt.xlim(-1, 1)
#plt.vlines(1420.4058, ymin=10e8, ymax=10e11, linewidth=.5)

# Madeline : Estimate Optimal Time Range for Galactic Plane Observation

In [ ]:
from astropy.coordinates import EarthLocation, SkyCoord

In [ ]:
# assumes we are at campbell
ugradio.timing.lst()

In [ ]:
# ideal HI viewing time is at LST = 19 - 6
galactic_data_source = np.load('../Data/Galaxy Data 1000.npz')

### Combine complex and real components and resize blocks

In [ ]:
galactic_data = galactic_data_source['arr_0'][0] - (galactic_data_source["arr_0"][1]*5/4*1j)

# resize blocks
galactic_data.shape = (-1, 2000)
galactic_data.shape

### Calculate average power spectrum 

In [ ]:
# GALACTIC DATA, S_ON BECAUSE WE ARE OBSERVING HYDROGEN LINE
galactic_freq = np.fft.fftfreq(galactic_data.shape[1], 1/62.5e6)
galactic_freq = np.fft.fftshift(galactic_freq)/1e6

blocks = 8000
avg_galactic_power_array = []
for n in range (0, blocks):
    data_new = galactic_data[n]
    ft = np.fft.fft(data_new)
    ft = np.fft.fftshift(ft)
    power = abs(ft)**2
    avg_galactic_power_array.append(power)
    
avg_galactic_power_array = np.mean(avg_galactic_power_array, axis = 0)
s_on = avg_galactic_power_array

### Plot average power spectrum

In [ ]:
#%matplotlib notebook
plt.figure(figsize=(15,8))
plt.plot(freq + 1230 + 190, avg_galactic_power_array, "purple", linewidth=2)
plt.yscale("log")
plt.xlabel("Frequency (MHz)", fontsize=15)
plt.ylabel("Power", fontsize=15)
plt.title("Average Power Spectrum of Galactic Data", fontsize=25)

#plt.xlim(-2, 2)
plt.vlines(1420.4058, ymin=0, ymax=10e11, linestyle="dashed", linewidth=.5)

# Mel : Calculate Gain

### find T_sys

In [ ]:
# temperature
T_rx = 300 #kelvin
T_sky = 10 #between 10-100K
T_sys = T_rx - T_sky
T_sys

### filter out spikes 

In [ ]:
# shape
g_freq = freq + 1230 + 190
s_line = s_on / s_cold #s_off=s_cold #GALACTIC POWER SPECTRA / SKY POWER SPECTRA #probably wrong. ratio of on observation (hydrogen there), over off observation (no H)

filtered_s_cold = np.array([s for s in s_cold if not (g_freq[list(s_cold).index(s)]<1390 or g_freq[list(s_cold).index(s)]>1451 or (g_freq[list(s_cold).index(s)]< 1421 and g_freq[list(s_cold).index(s)]>1419))])
filtered_s_cal = np.array([s for s in s_cal if not (g_freq[list(s_cal).index(s)]<1390 or g_freq[list(s_cal).index(s)]>1451 or (g_freq[list(s_cal).index(s)]< 1421 and g_freq[list(s_cal).index(s)]>1419))])
print(len(filtered_s_cold), len(filtered_s_cal))


## Plot Filtered S_cold and S_cal

In [ ]:
filtered_freq = np.fft.fftfreq(filtered_s_cold.shape[0], 1/62.5e6)
filtered_freq = np.fft.fftshift(filtered_freq)/1e6

#%matplotlib notebook
plt.figure(figsize=(15,8))
plt.plot(filtered_freq + 1230 + 190, filtered_s_cold, "purple", linewidth=2, label="s_on")
plt.plot(filtered_freq + 1230 + 190, filtered_s_cal, "blue", linewidth=2, label="s_off")
#plt.plot(filtered_freq + 1230 + 190, filtered_s_on, "blue", linewidth=2, label="s_off")
plt.yscale("log")
plt.xlabel("Frequency (MHz)", fontsize=15)
plt.ylabel("Power", fontsize=15)
plt.title("Power Spectrum of Filtered S_on and S_off", fontsize=25)
plt.legend()

#plt.xlim(-2, 2)
#plt.vlines(1420.4058, ymin=0, ymax=10e11, linestyle="dashed", linewidth=.5)

### calculate G and T_line

In [ ]:
G = T_sys/(np.sum(filtered_s_cal - filtered_s_cold)) * np.sum(filtered_s_cold)
T_line = s_line * G #power spectra in temperature units 

In [ ]:
print("Gain G = " + str(G))

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(g_freq, s_on, "r", label="s_on")
plt.plot(g_freq, s_cold, label="s_cold")
plt.plot(g_freq, T_line)
plt.yscale("log")